# HW: X-ray images classification
--------------------------------------

### Edited by Bar Goldner & Daria Hasin

Before you begin, open Mobaxterm and connect to triton with the user and password you were give with. Activate the environment `2ndPaper` and then type the command `pip install scikit-image`.

In this assignment you will be dealing with classification of 32X32 X-ray images of the chest. The image can be classified into one of four options: lungs (l), clavicles (c), and heart (h) and background (b). Even though those labels are dependent, we will treat this task as multiclass and not as multilabel. The dataset for this assignment is located on a shared folder on triton (`/MLdata/MLcourse/X_ray/'`).

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import numpy as np
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

from tensorflow.keras.initializers import Constant
from tensorflow.keras.datasets import fashion_mnist
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from skimage.io import imread

from skimage.transform import rescale, resize, downscale_local_mean
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


/home/smorandv/anaconda3/envs/2ndPaper-t2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/smorandv/anaconda3/envs/2ndPaper-t2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/smorandv/anaconda3/envs/2ndPaper-t2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/smorandv/anaconda3/

In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.2)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
def preprocess(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        src = imread(os.path.join(datapath, fn),1)
        img = resize(src,(32,32),order = 3)
        
        images[ii,:,:,0] = img
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    BaseImages = images
    BaseY = Y
    return BaseImages, BaseY

In [4]:
def preprocess_train_and_val(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        images[ii,:,:,0] = imread(os.path.join(datapath, fn),1)
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    return images, Y

In [5]:
#Loading the data for training and validation:
src_data = '/MLdata/MLcourse/X_ray/'
train_path = src_data + 'train'
val_path = src_data + 'validation'
test_path = src_data + 'test'
BaseX_train , BaseY_train = preprocess_train_and_val(train_path)
BaseX_val , BaseY_val = preprocess_train_and_val(val_path)
X_test, Y_test = preprocess(test_path)

In [6]:
keras.backend.clear_session()

### PART 1: Fully connected layers 
--------------------------------------

---
<span style="color:red">***Task 1:***</span> *NN with fully connected layers. 

Elaborate a NN with 2 hidden fully connected layers with 300, 150 neurons and 4 neurons for classification. Use ReLU activation functions for the hidden layers and He_normal for initialization. Don't forget to flatten your image before feedforward to the first dense layer. Name the model `model_relu`.*

---

In [7]:
#--------------------------Impelment your code here:-------------------------------------
n_layers = 300
n_classes = 4
input_shape = (32,32,1)

model_relu = Sequential()
model_relu.add(Flatten(input_shape=input_shape))

model_relu.add(Dense(n_layers, activation='relu', kernel_initializer='he_normal'))
model_relu.add(Dense(n_layers/2, activation='relu', kernel_initializer='he_normal'))

model_relu.add(Dense(n_classes))
model_relu.add(Activation('softmax'))

#----------------------------------------------------------------------------------------

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model_relu.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 300)               307500    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 604       
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


Compile the model with the optimizer above, accuracy metric and adequate loss for multiclass task. Train your model on the training set and evaluate the model on the testing set. Print the accuracy and loss over the testing set.

In [10]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')

# saving the initial weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_MLP_relu_task1.h5"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)
print('Saved initialized model at %s ' % model_path)

history = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
loss_and_metrics = model_relu.evaluate(X_test, Y_test, verbose=2)

# saving the final weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "final_weights_MLP_relu_25_epochs.h5"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)
print('Saved trained model at %s ' % model_path)

print()
print('The loss is:', np.round(loss_and_metrics[0], 2))
print('The accuracy is:', np.round(loss_and_metrics[1], 2))
#----------------------------------------------------------------------------------------

Saved initialized model at results/init_weigths_MLP_relu_task1.h5 
Train on 6474 samples, validate on 1728 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
6474/6474 [==============================] - 1s 168us/sample - loss: 1.3429 - acc: 0.3803 - val_loss: 1.2153 - val_acc: 0.5156
Epoch 2/25
6474/6474 [==============================] - 1s 98us/sample - loss: 1.1582 - acc: 0.5726 - val_loss: 1.0917 - val_acc: 0.6308
Epoch 3/25
6474/6474 [==============================] - 1s 95us/sample - loss: 1.0563 - acc: 0.6535 - val_loss: 1.0075 - val_acc: 0.6846
Epoch 4/25
6474/6474 [==============================] - 1s 94us/sample - loss: 0.9837 - acc: 0.7053 - val_loss: 0.9474 - val_acc: 0.7292
Epoch 5/25
6474/6474 [==============================] - 1s 99us/sample - loss: 0.9268 - acc: 0.7340 - val_loss: 0.8994 - val_acc: 0.7436
Epoch 6/25
6474/6474 [==============================] - 1s 104us/sample - loss: 0.8833 - acc: 0.7495 - val_loss: 0.8608 - val_acc: 0.7529
Epoch 7/25
64

---
<span style="color:red">***Task 2:***</span> *Activation functions.* 

**Change the activation functions to LeakyRelu or tanh or sigmoid. Name the new model `new_a_model`. Explain how it can affect the model.**

We changed the activation functions to tanh and it can effect the model in some ways:
* In tanh we can see the vanishing gradient problem - the gradient can be very small and prevent a weight from changing its value, while ReLU avoids and rectifies this problem.
* The proccess can be slower. ReLu is less computationally expensive than tanh because it involves simpler mathematical operations.

---

In [11]:
#--------------------------Impelment your code here:-------------------------------------
n_layers = 300
n_classes = 4
input_shape = (32,32,1)

new_a_model = Sequential()
new_a_model.add(Flatten(input_shape=input_shape))

new_a_model.add(Dense(n_layers, activation='tanh', kernel_initializer='he_normal'))
new_a_model.add(Dense(n_layers/2, activation='tanh', kernel_initializer='he_normal'))

new_a_model.add(Dense(n_classes))
new_a_model.add(Activation('softmax'))
#----------------------------------------------------------------------------------------

In [12]:
new_a_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               307500    
_________________________________________________________________
dense_4 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 604       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


---
<span style="color:red">***Task 3:***</span> *Number of epochs.* 

**Train the new model using 25 and 40 epochs. What difference does it makes in term of performance? Remember to save the compiled model for having initialized weights for every run as we did in tutorial 12. Evaluate each trained model on the test set**

In term of performance, greater number of epochs will get better results in the training set (high accuracy), but we will risk over-fitting which result in low accuracy on the training set. 

---

In [13]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

In [14]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')

# saving the initial weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_MLP_tanh_task3.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved initialized model at %s ' % model_path)

history = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
loss_and_metrics = new_a_model.evaluate(X_test, Y_test, verbose=2)

# saving the final weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "final_weights_MLP_tanh_25_epochs.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved trained model at %s ' % model_path)

print()
print('The loss is:', np.round(loss_and_metrics[0], 2))
print('The accuracy is:', np.round(loss_and_metrics[1], 2))
#-----------------------------------------------------------------------------------------

Saved initialized model at results/init_weigths_MLP_tanh_task3.h5 
Train on 6474 samples, validate on 1728 samples
Epoch 1/25
6474/6474 [==============================] - 1s 144us/sample - loss: 1.2664 - acc: 0.4425 - val_loss: 1.1626 - val_acc: 0.5689
Epoch 2/25
6474/6474 [==============================] - 1s 102us/sample - loss: 1.0991 - acc: 0.6095 - val_loss: 1.0506 - val_acc: 0.6221
Epoch 3/25
6474/6474 [==============================] - 1s 103us/sample - loss: 1.0106 - acc: 0.6498 - val_loss: 0.9830 - val_acc: 0.6649
Epoch 4/25
6474/6474 [==============================] - 1s 102us/sample - loss: 0.9497 - acc: 0.6736 - val_loss: 0.9332 - val_acc: 0.6916
Epoch 5/25
6474/6474 [==============================] - 1s 103us/sample - loss: 0.9064 - acc: 0.6945 - val_loss: 0.8967 - val_acc: 0.6979
Epoch 6/25
6474/6474 [==============================] - 1s 104us/sample - loss: 0.8694 - acc: 0.7045 - val_loss: 0.8655 - val_acc: 0.7141
Epoch 7/25
6474/6474 [==============================] - 1

In [15]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 40
# epochs = 50  # check for task 4: Batch normalization question.

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

In [16]:
#--------------------------Impelment your code here:-------------------------------------
from tensorflow.keras.models import load_model
new_a_model = load_model("results/init_weigths_MLP_tanh_task3.h5")

history = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
loss_and_metrics = new_a_model.evaluate(X_test, Y_test, verbose=2)

# saving the final weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "final_weights_MLP_tanh_40_epochs.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved trained model at %s ' % model_path)

print()
print('The loss is:', np.round(loss_and_metrics[0], 2))
print('The accuracy is:', np.round(loss_and_metrics[1], 2))
#-----------------------------------------------------------------------------------------

Train on 6474 samples, validate on 1728 samples
Epoch 1/40
6474/6474 [==============================] - 1s 140us/sample - loss: 1.2677 - acc: 0.4370 - val_loss: 1.1602 - val_acc: 0.5718
Epoch 2/40
6474/6474 [==============================] - 1s 106us/sample - loss: 1.1000 - acc: 0.6009 - val_loss: 1.0516 - val_acc: 0.6273
Epoch 3/40
6474/6474 [==============================] - 1s 101us/sample - loss: 1.0137 - acc: 0.6452 - val_loss: 0.9859 - val_acc: 0.6701
Epoch 4/40
6474/6474 [==============================] - 1s 101us/sample - loss: 0.9554 - acc: 0.6739 - val_loss: 0.9369 - val_acc: 0.6806
Epoch 5/40
6474/6474 [==============================] - 1s 101us/sample - loss: 0.9097 - acc: 0.6901 - val_loss: 0.9053 - val_acc: 0.6979
Epoch 6/40
6474/6474 [==============================] - 1s 103us/sample - loss: 0.8749 - acc: 0.7025 - val_loss: 0.8722 - val_acc: 0.7124
Epoch 7/40
6474/6474 [==============================] - 1s 104us/sample - loss: 0.8462 - acc: 0.7115 - val_loss: 0.8460 - va

---
<span style="color:red">***Task 4:***</span> *Mini-batches.* 

**Build the `model_relu` again and run it with a batch size of 32 instead of 64. What are the advantages of the mini-batch vs. SGD?**

The advantages of mini-batch over SGD are:
* Higher accuracy
* Better computational efficiency 
* Mini-batch perform weight updates more often than SGD, so we achieve faster learning.

---

In [17]:
keras.backend.clear_session()

In [18]:
#--------------------------Impelment your code here:-------------------------------------
n_layers = 300
n_classes = 4
input_shape = (32,32,1)

model_relu = Sequential()
model_relu.add(Flatten(input_shape=input_shape))

model_relu.add(Dense(n_layers, activation='relu', kernel_initializer='he_normal'))
model_relu.add(Dropout(0.2))

model_relu.add(Dense(n_layers/2, activation='relu', kernel_initializer='he_normal'))
model_relu.add(Dropout(0.2))

model_relu.add(Dense(n_classes))
model_relu.add(Activation('softmax'))
#----------------------------------------------------------------------------------------

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
batch_size = 32
epochs = 50
learn_rate = 1e-5
decay = 0

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


In [20]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')

# saving the initial weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_MLP_relu_task4.h5"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)
print('Saved initialized model at %s ' % model_path)

history = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
loss_and_metrics = model_relu.evaluate(X_test, Y_test, verbose=2)

# saving the final weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "final_weights_MLP_relu_50_epochs.h5"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)
print('Saved trained model at %s ' % model_path)

print()
print('The loss is:', np.round(loss_and_metrics[0], 2))
print('The accuracy is:', np.round(loss_and_metrics[1], 2))
#----------------------------------------------------------------------------------------

Saved initialized model at results/init_weigths_MLP_relu_task4.h5 
Train on 6474 samples, validate on 1728 samples
Epoch 1/50
6474/6474 [==============================] - 2s 256us/sample - loss: 1.3362 - acc: 0.3761 - val_loss: 1.1220 - val_acc: 0.6019
Epoch 2/50
6474/6474 [==============================] - 1s 218us/sample - loss: 1.1394 - acc: 0.5187 - val_loss: 1.0236 - val_acc: 0.6539
Epoch 3/50
6474/6474 [==============================] - 1s 214us/sample - loss: 1.0561 - acc: 0.5783 - val_loss: 0.9671 - val_acc: 0.6771
Epoch 4/50
6474/6474 [==============================] - 1s 209us/sample - loss: 0.9951 - acc: 0.6171 - val_loss: 0.9131 - val_acc: 0.7072
Epoch 5/50
6474/6474 [==============================] - 1s 226us/sample - loss: 0.9504 - acc: 0.6470 - val_loss: 0.8705 - val_acc: 0.7170
Epoch 6/50
6474/6474 [==============================] - 1s 216us/sample - loss: 0.9088 - acc: 0.6735 - val_loss: 0.8388 - val_acc: 0.7263
Epoch 7/50
6474/6474 [==============================] - 1

---
<span style="color:red">***Task 4:***</span> *Batch normalization.* 

**Build the `new_a_model` again and add batch normalization layers. How does it impact your results?**

Batch normalization is a technique that standardizes the inputs to a layer for each mini-batch. This way, the model achieves convergence faster and stabilizing the learning process.


We can see the impact on our results in the accuracy metric (for 50 epochs): 

With batch normalization: 0.6971

Without batch normalization: 0.6629

---

In [21]:
keras.backend.clear_session()

In [22]:
#--------------------------Impelment your code here:-------------------------------------
n_layers = 300
n_classes = 4
input_shape = (32,32,1)

new_a_model = Sequential()
new_a_model.add(Flatten(input_shape=input_shape))

new_a_model.add(Dense(n_layers, activation='tanh', kernel_initializer='he_normal'))
new_a_model.add(BatchNormalization())
new_a_model.add(Dropout(0.2))

new_a_model.add(Dense(n_layers/2, activation='tanh', kernel_initializer='he_normal'))
new_a_model.add(BatchNormalization())
new_a_model.add(Dropout(0.2))

new_a_model.add(Dense(n_classes))
new_a_model.add(Activation('softmax'))
#---------------------------------------------------------------------------------------

In [23]:
batch_size = 32
epochs = 50
learn_rate = 1e-5
decay = 0

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)
#Compile the network: 
new_a_model.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')

In [24]:
#Preforming the training by using fit 
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')

# saving the initial weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_MLP_tanh_task4.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved initialized model at %s ' % model_path)

history = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
loss_and_metrics = new_a_model.evaluate(X_test, Y_test, verbose=2)

# saving the final weights
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "final_weights_MLP_tanh_50_epochs_batch_norm.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved trained model at %s ' % model_path)

print()
print('The loss is:', np.round(loss_and_metrics[0], 2))
print('The accuracy is:', np.round(loss_and_metrics[1], 2))
#----------------------------------------------------------------------------------------

Saved initialized model at results/init_weigths_MLP_tanh_task4.h5 
Train on 6474 samples, validate on 1728 samples
Epoch 1/50
6474/6474 [==============================] - 2s 371us/sample - loss: 1.4114 - acc: 0.4568 - val_loss: 1.0204 - val_acc: 0.5961
Epoch 2/50
6474/6474 [==============================] - 2s 311us/sample - loss: 1.0474 - acc: 0.5947 - val_loss: 0.7925 - val_acc: 0.7188
Epoch 3/50
6474/6474 [==============================] - 2s 306us/sample - loss: 0.9327 - acc: 0.6413 - val_loss: 0.7081 - val_acc: 0.7471
Epoch 4/50
6474/6474 [==============================] - 2s 309us/sample - loss: 0.8498 - acc: 0.6773 - val_loss: 0.6399 - val_acc: 0.7778
Epoch 5/50
6474/6474 [==============================] - 2s 299us/sample - loss: 0.7875 - acc: 0.6959 - val_loss: 0.6129 - val_acc: 0.7940
Epoch 6/50
6474/6474 [==============================] - 2s 302us/sample - loss: 0.7469 - acc: 0.7206 - val_loss: 0.5677 - val_acc: 0.8056
Epoch 7/50
6474/6474 [==============================] - 2

### PART 2: Convolutional Neural Network (CNN)
------------------------------------------------------------------------------------

---
<span style="color:red">***Task 1:***</span> *2D CNN.* 

Have a look at the model below and answer the following:

**How many layers does it have?**
    
The model has 8 layers: 5 layers of 2D convolution and 3 layers of fully connected.
    
    
**How many filter in each layer?**
    
Conv2D_1: 64 filters

Conv2D_2: 128 filters

Conv2D_3: 128 filters

Conv2D_4: 256 filters

Conv2D_5: 256 filters
    

**Would the number of parmaters be similar to a fully connected NN?**
    
No, the number will be different because in a fully connected NN each neuron is connected to every neuron in the previous layer, and each connection has it's own weight (trainable parameter), while in 2D convolution each neuron is only connected to a few neurons in the previous layer, and the same set of weights is used for every neuron. 


**Is this specific NN performing regularization?**

Yes, the NN performing regularization:
* Each 2D convolution layer include regularization kernel (L2).
* If drop is True, we have 6 dropouts in the flow. Dropout is a regularization technique that zeros weights of some neurons in the layer randomly (the rate is defined by dropRate).

---

In [25]:
def get_net(input_shape,drop,dropRate,reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

In [26]:
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet = get_net(input_shape,drop,dropRate,reg)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_v1_2 (Ba (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 128)       0         
__________

In [27]:
NNet=get_net(input_shape,drop,dropRate,reg)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_8 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_v1_7 (Ba (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_9 (Dropout)          (None, 16, 16, 128)       0         
__________

In [28]:
from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
# Checkpath = os.getcwd()
# Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

In [29]:
#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet.save(model_fn)

Train on 6474 samples, validate on 1728 samples
Epoch 1/25
6474/6474 [==============================] - 5s 769us/sample - loss: 8.0198 - acc: 0.4521 - val_loss: 7.8581 - val_acc: 0.2500
Epoch 2/25
6474/6474 [==============================] - 2s 351us/sample - loss: 7.5996 - acc: 0.5428 - val_loss: 8.0109 - val_acc: 0.2529
Epoch 3/25
6474/6474 [==============================] - 2s 352us/sample - loss: 7.4377 - acc: 0.5879 - val_loss: 8.0771 - val_acc: 0.2454
Epoch 4/25
6474/6474 [==============================] - 2s 351us/sample - loss: 7.3416 - acc: 0.6182 - val_loss: 7.9738 - val_acc: 0.2865
Epoch 5/25
6474/6474 [==============================] - 2s 347us/sample - loss: 7.2284 - acc: 0.6484 - val_loss: 7.8199 - val_acc: 0.2992
Epoch 6/25
6474/6474 [==============================] - 2s 344us/sample - loss: 7.1604 - acc: 0.6630 - val_loss: 7.6127 - val_acc: 0.3981
Epoch 7/25
6474/6474 [==============================] - 2s 346us/sample - loss: 7.0866 - acc: 0.6946 - val_loss: 7.5301 - va

In [30]:
# NNet.load_weights('Weights_1.h5')

In [31]:
results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

175/175 [==============================] - 0s 471us/sample - loss: 7.7784 - acc: 0.3200
test loss, test acc: [7.7783691024780275, 0.32]


---
<span style="color:red">***Task 2:***</span> *Number of filters* 

Rebuild the function `get_net` to have as an input argument a list of number of filters in each layers, i.e. for the CNN defined above the input should have been `[64, 128, 128, 256, 256]`. Now train the model with the number of filters reduced by half. What were the results.

---

In [ ]:
#--------------------------Impelment your code here:-------------------------------------
def get_net(filters_list, input_shape, drop, dropRate, reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=filters_list[0], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filters_list[1], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filters_list[2], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filters_list[3], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filters_list[4], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model


input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
filters = [32, 64, 64, 128, 128]

NNet = get_net(filters, input_shape, drop, dropRate, reg)

AdamOpt = Adam(lr=learn_rate,decay=decay)
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

history = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
loss_and_metrics = NNet.evaluate(X_test,Y_test)
print()
print('The loss is:', np.round(loss_and_metrics[0], 2))
print('The accuracy is:', np.round(loss_and_metrics[1], 2))

#----------------------------------------------------------------------------------------

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
dropout_14 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_v1_12 (B (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_15 (Dropout)         (None, 16, 16, 64)        0         
__________

That's all folks! See you :)